<a href="https://colab.research.google.com/github/joanadecaa1/data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: date - yyyy-mm-dd or yyyymmdd)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [ ]:
%pip install pyspark

In [40]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests

class ETLFlow:
    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract_from_file(self, format: str, path: str, **kwargs) -> DataFrame:
        df = self.spark.read.format(format).load(path)
        print(f"Extracted data from {path}:")
        df.show(5, truncate=False)  # Display the first 5 rows for inspection
        return df

    def extract_from_api(self, url: str, schema: StructType = None):
        response = requests.get(url)
        rdd = self.spark.sparkContext.parallelize(response.json())
        if schema:
            df = self.spark.read.schema(schema).json(rdd)
        else:
            df = self.spark.read.json(rdd)
        print(f"Extracted data from API {url}:")
        df.show(5, truncate=False)  # Display the first 5 rows for inspection
        return df

    def load(self, df: DataFrame, format: str, path: str, partition_column: str = None, **kwargs) -> None:
        print(f"Loading data to {path}...")
        if partition_column:
            df.coalesce(1).write.mode("overwrite").partitionBy(partition_column).format(format).save(path)
        else:
            df.coalesce(1).write.mode("overwrite").format(format).save(path)
        print(f"Data successfully loaded to {path}")

class ETLTask(ETLFlow):
    def __init__(self, spark: SparkSession) -> None:
        super().__init__(spark)

    def ingestion_lines(self):
        # Define schema for lines
        lines_schema = StructType([
            StructField('color', StringType(), True),
            StructField('facilities', ArrayType(StringType(), True), True),
            StructField('id', StringType(), True),
            StructField('localities', ArrayType(StringType(), True), True),
            StructField('long_name', StringType(), True),
            StructField('municipalities', ArrayType(StringType(), True), True),
            StructField('patterns', ArrayType(StringType(), True), True),
            StructField('routes', ArrayType(StringType(), True), True),
            StructField('short_name', StringType(), True),
            StructField('text_color', StringType(), True)
        ])
        # Ingest data
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/lines", schema=lines_schema)
        # Load into Bronze layer
        self.load(df=df, format="parquet", path="/content/lake/bronze/lines")

    def ingestion_vehicles(self):
        # Define schema for vehicles
        vehicle_schema = StructType([
            StructField('bearing', IntegerType(), True),
            StructField('block_id', StringType(), True),
            StructField('current_status', StringType(), True),
            StructField('id', StringType(), True),
            StructField('lat', FloatType(), True),
            StructField('line_id', StringType(), True),
            StructField('lon', FloatType(), True),
            StructField('pattern_id', StringType(), True),
            StructField('route_id', StringType(), True),
            StructField('schedule_relationship', StringType(), True),
            StructField('shift_id', StringType(), True),
            StructField('speed', FloatType(), True),
            StructField('stop_id', StringType(), True),
            StructField('timestamp', TimestampType(), True),
            StructField('trip_id', StringType(), True)
        ])
        # Ingest data
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)
        print("Raw data from API - Vehicles:")
        df.show(5, truncate=False)  # Display the first 5 rows of raw data

        # Add date column
        df = df.withColumn("date", expr("date(timestamp)"))
        print("Data after adding date column:")
        df.show(5, truncate=False)  # Display data after adding date column

        # Load into Bronze layer, partitioned by date
        self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles", partition_column="date")

    def ingestion_municipalities(self):
        # Define schema for municipalities (updated)
        municipalities_schema = StructType([
            StructField('id', StringType(), True),
            StructField('name', StringType(), True),
            StructField('district_id', StringType(), True),  # Renamed to match API response
            StructField('district_name', StringType(), True),  # New field added
            StructField('prefix', StringType(), True),
            StructField('region_id', StringType(), True),
            StructField('region_name', StringType(), True)
        ])
        # Ingest data
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/municipalities", schema=municipalities_schema)
        print("Raw data from API - Municipalities:")
        df.show(5, truncate=False)  # Display the first 5 rows of raw data

        # Add ingestion date
        df = df.withColumn("ingestion_date", current_date())
        print("Data after adding ingestion date:")
        df.show(5, truncate=False)  # Display data after adding ingestion date

        # Load into Bronze layer
        self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")

    def cleansing_vehicles(self):
        # Read raw data
        df = self.extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")
        print("Raw data from Bronze - Vehicles:")
        df.show(5, truncate=False)  # Display the first 5 rows of raw data

        # Transformations
        df = df.withColumn("new_column", lit("test"))
        print("Data after adding new column:")
        df.show(5, truncate=False)  # Display data after adding new column

        df = df.drop_duplicates()
        print("Data after removing duplicates:")
        df.show(5, truncate=False)  # Display data after removing duplicates

        # Load into Silver layer
        self.load(df=df, format="parquet", path="/content/lake/silver/vehicles")

if __name__ == '__main__':
    # Initialize SparkSession
    spark = SparkSession.builder.master('local').appName('ETL Program').getOrCreate()
    print("Starting ETL program")
    etl = ETLTask(spark)

    # Run tasks
    print("Running Task - Ingestion Vehicles")
    etl.ingestion_vehicles()

    print("Running Task - Ingestion Lines")
    etl.ingestion_lines()

    print("Running Task - Ingestion Municipalities")
    etl.ingestion_municipalities()

    print("Running Task - Cleansing Vehicles")
    etl.cleansing_vehicles()

    print("ETL program completed")


Starting ETL program
Running Task - Ingestion Vehicles
Extracted data from API https://api.carrismetropolitana.pt/vehicles:
+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+----------------------------+
|bearing|block_id                      |current_status|id      |lat      |line_id|lon      |pattern_id|route_id|schedule_relationship|shift_id    |speed    |stop_id|timestamp          |trip_id                     |
+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+----------------------------+
|348    |20241126-64010339-112550234560|IN_TRANSIT_TO |44|12061|38.5271  |4422   |-8.902385|4422_1_1  |4422_1  |SCHEDULED            |112550234560|0.0      |160438 |2024-11-26 23:33:49|4422_1_1|2600|2315_24UH5    |
|52     |2024112

-------------------------

In [62]:
import os

# Create the directory paths
os.makedirs('/content/lake/silver/vehicles', exist_ok=True)
os.makedirs('/content/lake/silver/lines', exist_ok=True)
os.makedirs('/content/lake/silver/municipalities', exist_ok=True)


In [63]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Cleansing Process") \
    .getOrCreate()

# Paths for the Bronze layer
vehicles_path = '/content/lake/bronze/vehicles'
lines_path = '/content/lake/bronze/lines'
municipalities_path = '/content/lake/bronze/municipalities'

# Read data from the Bronze layer
vehicles_df = spark.read.parquet(vehicles_path)
lines_df = spark.read.parquet(lines_path)
municipalities_df = spark.read.parquet(municipalities_path)


In [64]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Cleansing Process") \
    .getOrCreate()

# Paths for the Bronze layer
vehicles_path = '/content/lake/bronze/vehicles'
lines_path = '/content/lake/bronze/lines'
municipalities_path = '/content/lake/bronze/municipalities'

# Read data from the Bronze layer
vehicles_df = spark.read.parquet(vehicles_path)
lines_df = spark.read.parquet(lines_path)
municipalities_df = spark.read.parquet(municipalities_path)


In [65]:
# Rename columns
vehicles_df = vehicles_df.withColumnRenamed("lat", "latitude").withColumnRenamed("lon", "longitude")

# Remove duplicates
vehicles_df = vehicles_df.dropDuplicates()

# Remove rows with null CURRENT_STATUS
vehicles_df = vehicles_df.filter(vehicles_df["CURRENT_STATUS"].isNotNull())

# Remove any corrupted records (e.g., rows with null or malformed data)
# Assuming all columns should be non-null for simplicity
vehicles_df = vehicles_df.dropna()


In [66]:
# Remove duplicates
lines_df = lines_df.dropDuplicates()

# Remove rows with null LONG_NAME
lines_df = lines_df.filter(lines_df["LONG_NAME"].isNotNull())

# Remove any corrupted records
lines_df = lines_df.dropna()


In [67]:
# Remove duplicates
municipalities_df = municipalities_df.dropDuplicates()

# Remove rows where NAME or DISTRICT_NAME is null
municipalities_df = municipalities_df.filter(
    (municipalities_df["NAME"].isNotNull()) &
    (municipalities_df["DISTRICT_NAME"].isNotNull())
)

# Remove any corrupted records
municipalities_df = municipalities_df.dropna()


In [68]:
# Write transformed data to the Silver layer
vehicles_path_silver = '/content/lake/silver/vehicles'
lines_path_silver = '/content/lake/silver/lines'
municipalities_path_silver = '/content/lake/silver/municipalities'

# Write vehicles data partitioned by 'date'

vehicles_df.write.partitionBy("date").mode("overwrite").parquet(vehicles_path_silver)


# Write lines data
lines_df.write.mode("overwrite").parquet(lines_path_silver)

# Write municipalities data
municipalities_df.write.mode("overwrite").parquet(municipalities_path_silver)


In [69]:
# Read data from the Silver layer
vehicles_silver_df = spark.read.parquet('/content/lake/silver/vehicles')
lines_silver_df = spark.read.parquet('/content/lake/silver/lines')
municipalities_silver_df = spark.read.parquet('/content/lake/silver/municipalities')

# Display the first few rows of each DataFrame
print("Vehicles Data:")
vehicles_silver_df.show(5)

print("\nLines Data:")
lines_silver_df.show(5)

print("\nMunicipalities Data:")
municipalities_silver_df.show(5)


Vehicles Data:
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|      date|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+
|      0|           1_1019-11| IN_TRANSIT_TO| 41|1251|38.800488|   1517|-9.233361|  1517_0_1|  1517_0|            SCHEDULED|        1134|      0.0| 172169|2024-11-26 23:33:36|1517_0_1_2330_235...|2024-11-26|
|     20|             1185-11|   INCOMING_AT| 42|2370|  38.9547|   2328| -8.98699|  2328_0_1|  2328_0|            SCHEDULED|        1279| 6.111111| 18076

----------------------